In [1]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [2]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [3]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [4]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [5]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_market_data)

In [6]:
# Create a DataFrame with the scaled data
df_scaled = pd.DataFrame(scaled_data, columns=df_market_data.columns, index=df_market_data.index)

# Copy the crypto names from the original data
crypto_names = df_market_data.index
# Set the coinid column as index

df_market_data.reset_index(inplace=True)
# Display sample data
df_market_data.head()

,coin_id,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
0,bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
1,ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
2,tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
3,ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
4,bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384


---

### Find the Best Value for k Using the Original Data.

In [7]:
# Create a list with the number of k-values from 1 to 11
k_values = list(range(1, 12))

In [8]:
# Create an empty list to store the inertia values
inertia_values = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
for k in range(1, 12):
    kmeans = KMeans(n_clusters=k)
    
    kmeans.fit(df_scaled)  
    
   
    inertia_values.append(kmeans.inertia_)
    

c:\Users\kirah\anaconda3\envs\bootcamp\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\kirah\anaconda3\envs\bootcamp\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\kirah\anaconda3\envs\bootcamp\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\kirah\anaconda3\envs\bootcamp\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known t

In [9]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {
    'k_values': list(range(1, 12)),
    'inertia_values': inertia_values
}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow = pd.DataFrame(elbow_data)

df_elbow


,k_values,inertia_values
0,1,287.000000
1,2,195.820218
2,3,123.190482
3,4,79.022435
4,5,65.284985
5,6,54.739922
6,7,43.989781
7,8,37.284807
8,9,32.646297
9,10,28.065823


In [10]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_plot = df_elbow.hvplot.line(x='k_values', 
                                  y='inertia_values', 
                                  xlabel='Number of Clusters (k)',
                                  ylabel='Inertia', 
                                  title='Elbow Curve')
elbow_plot

:Curve   [k_values]   (inertia_values)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:**  Best Value for K is 4

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [11]:
# Initialize the K-Means model using the best value for k
kmeans = KMeans(n_clusters=4)


In [12]:
# Fit the K-Means model using the scaled data
kmeans.fit(df_scaled)

c:\Users\kirah\anaconda3\envs\bootcamp\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\kirah\anaconda3\envs\bootcamp\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


KMeans(n_clusters=4)

In [13]:
# Predict the clusters to group the cryptocurrencies using the scaled data
cluster_labels = kmeans.predict(df_scaled)

# Print the resulting array of cluster values.
cluster_labels

array([3, 3, 0, 0, 3, 3, 3, 3, 3, 0, 0, 0, 0, 3, 0, 3, 0, 0, 3, 0, 0, 3,
       0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 1, 3, 0, 0, 2, 0, 0, 0, 0])

In [14]:
# Create a copy of the DataFrame
df_copy = df_market_data.copy()

In [15]:
# Add a new column to the DataFrame with the predicted clusters
df_copy['predicted_cluster'] = cluster_labels

# Display sample data
df_copy.head()

,coin_id,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,predicted_cluster
0,bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761,3
1,ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023,3
2,tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954,0
3,ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193,0
4,bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384,3


In [16]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.

scatter_plot = df_copy.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    c="predicted_cluster",
    colormap="viridis", 
    hover_cols=["coin_id"],
    xlabel="Price Change Percentage (24h)",
    ylabel="Price Change Percentage (7d)",
    title="Cryptocurrency Clusters"
)

# Display the scatter plot
scatter_plot


:Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,predicted_cluster,coin_id)

---

### Optimize Clusters with Principal Component Analysis.

In [17]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [18]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.

principal_components = pca.fit_transform( df_scaled)

df_principal_components = pd.DataFrame(
    data=principal_components,
    columns=['PC1', 'PC2', 'PC3'],
    index=df_market_data.index  
)
df_principal_components ['coin_id'] = df_scaled.index
# View the first five rows of the DataFrame. 
df_principal_components.head()

,PC1,PC2,PC3,coin_id
0,-0.600667,0.842760,0.461595,bitcoin
1,-0.458261,0.458466,0.952877,ethereum
2,-0.433070,-0.168126,-0.641752,tether
3,-0.471835,-0.222660,-0.479053,ripple
4,-1.157800,2.041209,1.859715,bitcoin-cash


In [19]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
EV = pca.explained_variance_ratio_
EV

array([0.3719856 , 0.34700813, 0.17603793])

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** The total explained variance of the three principal components is approximately 89.8%.  That means that most of the essential information from the original data is retained in the reduced-dimensional space defined by these principal components.

In [22]:
# Create a new DataFrame with the PCA data.

# Creating a DataFrame with the PCA data 
df_pca = pd.DataFrame(
    data=principal_components,
    columns=['Principal Component 1', 'Principal Component 2', 'Principal Component 3'],
    index=df_market_data.index 
)
# Copy the cryptocurrency names from the original data
df_pca['coin_id'] = df_market_data.index

# Set the 'coin_id' column as index
df_pca.set_index('coin_id', inplace=True)


# Display sample data
df_pca.head()

,Principal Component 1,Principal Component 2,Principal Component 3
coin_id,,,
0,-0.600667,0.842760,0.461595
1,-0.458261,0.458466,0.952877
2,-0.433070,-0.168126,-0.641752
3,-0.471835,-0.222660,-0.479053
4,-1.157800,2.041209,1.859715


---

### Find the Best Value for k Using the PCA Data

In [27]:
# Create a list with the number of k-values from 1 to 11
list_k_value = list(range(1, 12))

In [28]:
# Create an empty list to store the inertia values
inertia_values = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
for k in list_k_value:
    # 1. Create a KMeans model using the loop counter for the n_clusters
    kmeans = KMeans(n_clusters=k, random_state=0)
    
    # 2. Fit the model to the data using df_pca
    kmeans.fit(df_pca)
    
    # 3. Append the model.inertia_ to the inertia list
    inertia_values.append(kmeans.inertia_)


c:\Users\kirah\anaconda3\envs\bootcamp\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\kirah\anaconda3\envs\bootcamp\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\kirah\anaconda3\envs\bootcamp\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\kirah\anaconda3\envs\bootcamp\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known t

In [29]:
# Create a dictionary with the data to plot the Elbow curve
C_elbow_data = {
    'list_k_value': list_k_value,
    'inertia_values': inertia_values
}

# Create a DataFrame with the data to plot the Elbow curve
C_df_elbow = pd.DataFrame(C_elbow_data)

# Display the DataFrame to verify
print(C_df_elbow)

    list_k_value  inertia_values
0              1      256.874086
1              2      168.811896
2              3       93.774626
3              4       49.665497
4              5       37.839466
5              6       27.720626
6              7       21.182776
7              8       17.016962
8              9       13.668325
9             10       10.484890
10            11        7.984355


In [39]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_plot_pca= C_df_elbow.hvplot.line(
    x='list_k_value',
    y='inertia_values',
    title='Elbow Curve',
    xlabel='Number of Clusters (k)',
    line_color= 'red',
    ylabel='Inertia',
    width=800,
    height=400
)

elbow_plot_pca

:Curve   [list_k_value]   (inertia_values)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** The best value for k is 4.


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** No, the best value for k is the same for both the PCA data and the original data, which is 4.

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [31]:
# Initialize the K-Means model using the best value for k
kmeans_pca = KMeans(n_clusters=4, random_state=0)

In [32]:
# Fit the K-Means model using the PCA data
kmeans_pca.fit(df_pca)

c:\Users\kirah\anaconda3\envs\bootcamp\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\kirah\anaconda3\envs\bootcamp\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


KMeans(n_clusters=4, random_state=0)

In [33]:
# Predict the clusters to group the cryptocurrencies using the PCA data
pca_clusters = kmeans_pca.predict(df_pca)
# Print the resulting array of cluster values.
pca_clusters

array([2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 2,
       0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 2, 0, 0, 3, 0, 0, 0, 0])

In [34]:
# Create a copy of the DataFrame with the PCA data
df_pca_copy = df_pca.copy()

# Add a new column to the DataFrame with the predicted clusters
df_pca_copy['predicted_cluster'] = pca_clusters

# Display sample data
df_pca_copy.head()


,Principal Component 1,Principal Component 2,Principal Component 3,predicted_cluster
coin_id,,,,
0,-0.600667,0.842760,0.461595,2
1,-0.458261,0.458466,0.952877,2
2,-0.433070,-0.168126,-0.641752,0
3,-0.471835,-0.222660,-0.479053,0
4,-1.157800,2.041209,1.859715,2


In [37]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
scatter_plot_pca = df_pca_copy.hvplot.scatter(
    x='Principal Component 1',  
    y='Principal Component 2', 
    c='predicted_cluster', 
    colormap='viridis',
    hover_cols=['coin_id'],
    xlabel='Principal Component 1 (PC1)',
    ylabel='Principal Component 2 (PC2)',
    title='Cryptocurrency Clusters PCA'
)
scatter_plot_pca

:Scatter   [Principal Component 1]   (Principal Component 2,predicted_cluster,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [40]:
# Composite plot to contrast the Elbow curves
composite_plot = (elbow_plot * elbow_plot_pca).opts(
    title='Elbow Curve Comparison',
    xlabel='Number of Clusters (k)',
    ylabel='Inertia',
    width=800,
    height=400
)
composite_plot

:Overlay
   .Curve.I  :Curve   [list_k_value]   (inertia_values)
   .Curve.II :Curve   [list_k_value]   (inertia_values)

In [41]:
# Composite plot to contrast the clusters
# YOUR CODE HERE!
composite_plot_clusters = (scatter_plot + scatter_plot_pca).cols(2)

composite_plot_clusters

:Layout
   .Scatter.I  :Scatter   [Principal Component 1]   (Principal Component 2,predicted_cluster,coin_id)
   .Scatter.II :Scatter   [Principal Component 1]   (Principal Component 2,predicted_cluster,coin_id)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** Using fewer features, such as through PCA, simplifies the clustering process by reducing computational complexity and aiding visualization. However, it can also lead to a loss of detailed information present in the original dataset, potentially affecting the accuracy of clustering results